In [1]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 1.5MB 9.4MB/s 
     |████████████████████████████████| 890kB 17.2MB/s 
     |████████████████████████████████| 2.9MB 34.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=92fc087c23c7557f9f30e38680f6a4242afb34f31e9e0111d123663cc88ac108
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime


#https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification

In [3]:
# 네이버 영화리뷰 감정분석 데이터 다운로드
#박은정님의 네이버 영화리뷰 감정분석 데이터를 Github에서 다운로드 
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [4]:
# 디렉토리의 파일 목록
!ls nsmc -la

total 38636
drwxr-xr-x 5 root root     4096 Dec 27 06:20 .
drwxr-xr-x 1 root root     4096 Dec 27 06:20 ..
drwxr-xr-x 2 root root     4096 Dec 27 06:20 code
drwxr-xr-x 8 root root     4096 Dec 27 06:20 .git
-rw-r--r-- 1 root root  4893335 Dec 27 06:20 ratings_test.txt
-rw-r--r-- 1 root root 14628807 Dec 27 06:20 ratings_train.txt
-rw-r--r-- 1 root root 19515078 Dec 27 06:20 ratings.txt
drwxr-xr-x 2 root root   458752 Dec 27 06:20 raw
-rw-r--r-- 1 root root     2596 Dec 27 06:20 README.md
-rw-r--r-- 1 root root    36746 Dec 27 06:20 synopses.json


In [5]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [6]:
# 훈련셋의 앞부분 출력
train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [7]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                원작의 긴장감을 제대로 살려내지못했다.
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9        왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
Name: document, dtype: object

In [8]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

In [9]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 1, 0, ..., 0, 1, 0])

In [10]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


In [11]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [12]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [13]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9711,  11489,   9364,  41850,   9004,  32537,   9491,  35506,
         17360,  48549,    119,    119,   9477,  26444,  12692,   9665,  21789,
         11287,   9708, 119235,   9659,  22458, 119136,  12965,  48549,    119,
           119,   9532,  22879,   9685,  16985,  14523,  48549,    119,    119,
          9596, 118728,    119,    119,   9178, 106065, 118916,    119,    119,
          8903,  11664,  11513,   9960,  14423,  25503, 118671,  48549,    119,
           119,  21890,   9546,  37819,  22879,   9356,  14867,   9715, 119230,
        118716,  48345,    119,   9663,  23321,  10954,   9638,  35506, 106320,
         10739,  20173,   9359,  19105,  11102,  42428,  17196,  48549,    119,
           119,    100,    117,   9947,  12945,   9532,  25503,   8932,  14423,
         35506, 119050,  11903,  14867,  10003,  14863,  33188,  48345,    119,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [14]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [15]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object

In [16]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

In [17]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0])

In [18]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 굳 ㅋ [SEP]
['[CLS]', '굳', '[UNK]', '[SEP]']


In [19]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [20]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [21]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(1)
tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [22]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [23]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [24]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [25]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [26]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [27]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [28]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [29]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:06:01.
  Batch 1,000  of  4,219.    Elapsed: 0:12:07.
  Batch 1,500  of  4,219.    Elapsed: 0:18:12.
  Batch 2,000  of  4,219.    Elapsed: 0:24:18.
  Batch 2,500  of  4,219.    Elapsed: 0:30:24.
  Batch 3,000  of  4,219.    Elapsed: 0:36:29.
  Batch 3,500  of  4,219.    Elapsed: 0:42:34.
  Batch 4,000  of  4,219.    Elapsed: 0:48:40.

  Average training loss: 0.38
  Training epcoh took: 0:51:20

Running Validation...
  Accuracy: 0.86
  Validation took: 0:01:58

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:06:05.
  Batch 1,000  of  4,219.    Elapsed: 0:12:11.
  Batch 1,500  of  4,219.    Elapsed: 0:18:16.
  Batch 2,000  of  4,219.    Elapsed: 0:24:21.
  Batch 2,500  of  4,219.    Elapsed: 0:30:27.
  Batch 3,000  of  4,219.    Elapsed: 0:36:32.
  Batch 3,500  of  4,219.    Elapsed: 0:42:37.
  Batch 4,000  of  4,219.    Elapsed: 0:48:43.

  Average training loss: 0.29
  

In [30]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:25.
  Batch   200  of  1,563.    Elapsed: 0:00:50.
  Batch   300  of  1,563.    Elapsed: 0:01:15.
  Batch   400  of  1,563.    Elapsed: 0:01:40.
  Batch   500  of  1,563.    Elapsed: 0:02:05.
  Batch   600  of  1,563.    Elapsed: 0:02:30.
  Batch   700  of  1,563.    Elapsed: 0:02:55.
  Batch   800  of  1,563.    Elapsed: 0:03:20.
  Batch   900  of  1,563.    Elapsed: 0:03:45.
  Batch 1,000  of  1,563.    Elapsed: 0:04:10.
  Batch 1,100  of  1,563.    Elapsed: 0:04:35.
  Batch 1,200  of  1,563.    Elapsed: 0:05:00.
  Batch 1,300  of  1,563.    Elapsed: 0:05:25.
  Batch 1,400  of  1,563.    Elapsed: 0:05:50.
  Batch 1,500  of  1,563.    Elapsed: 0:06:15.

Accuracy: 0.87
Test took: 0:06:31


In [31]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks



# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits


In [32]:
logits = test_sentences(['내 문장 삽입 하기!!!!!!!!!!!!!!!'])

print(logits)
print(np.argmax(logits))

[[-1.2268453  1.1435305]]
1


In [33]:
import pandas as pd
import os
import numpy as np

In [34]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
%cd /content/drive/My\ Drive/TweetsMoon/

/content/drive/My Drive/TweetsMoon


In [36]:
# working directory check
print(os.getcwd())

/content/drive/My Drive/TweetsMoon


In [37]:
#read data
df = pd.read_csv("final_pre.csv", engine = 'python', names= ('Date', 'Pos','Neg', 'Tweet_date', 'text', 'favorites', 'retweets' ))
df

,Date,Pos,Neg,Tweet_date,text,favorites,retweets
0,2017년 5월 3주,81.60%,10.10%,2017-05-14,"찡찡이가 입주했는데,걱정이 생겼네요. 관저 구석의 유리창문과 미닫이 한지창문 사이의...",17444.250000,12855.250000
1,2017년 5월 4주,84.10%,10.00%,2017-05-21,오늘 이 추도식에 대통령으로 참석하겠다고 한 약속을 지킬 수 있게 해주신 것에 대해...,16838.625000,14297.500000
2,2017년 5월 5주,78.10%,14.20%,2017-05-28,바다의 날 기념식은 여러모로 의미 깊었습니다. 14년 만에 대통령이 참석해 해양수산...,4277.500000,6269.000000
3,2017년 6월 1주,78.90%,14.70%,2017-06-04,영국 런던에서 일어난 연쇄테러는 무고한 민간인과 어린이를 상대로 저질러진 범죄행위입...,5432.250000,6121.750000
4,2017년 6월 2주,75.60%,17.40%,2017-06-11,트위터의 잭 도시 님 감사합니다. 앞으로도 트위터를 통해 국민들과 즐겁게 소통하고 ...,3397.666667,4715.666667
...,...,...,...,...,...,...,...
157,2020년 6월 2주,58.20%,37.00%,2020-05-17,"서로 믿고 의지하며, 방역에서 세계를 선도하는 대한민국의 자긍심을 계속 이어나가길 ...",2703.956191,5205.188571
158,2020년 6월 3주,53.40%,41.80%,2020-05-24,"재정이 경제 충격의 파고를 막는 방파제, 경제 회복을 앞당기는 마중물 역할을 해야 ...",2796.750000,5618.750000
159,2020년 6월 4주,53.30%,42.70%,2020-05-31,오늘 비상경제회의에서 포스트 코로나 시대를 앞서가기 위한 국가프로젝트로서 한국판 뉴...,3243.000000,5991.055555
160,2020년 7월 1주,49.80%,45.50%,2020-06-07,"할머니들도 그런 마음이실 겁니다. 협성양로원 할머니들처럼 조심하는 마음들이 있다면,...",2671.166667,5200.854167


In [56]:
text = df.text
text

0      찡찡이가 입주했는데,걱정이 생겼네요. 관저 구석의 유리창문과 미닫이 한지창문 사이의...
1      오늘 이 추도식에 대통령으로 참석하겠다고 한 약속을 지킬 수 있게 해주신 것에 대해...
2      바다의 날 기념식은 여러모로 의미 깊었습니다. 14년 만에 대통령이 참석해 해양수산...
3      영국 런던에서 일어난 연쇄테러는 무고한 민간인과 어린이를 상대로 저질러진 범죄행위입...
4      트위터의 잭 도시 님 감사합니다. 앞으로도 트위터를 통해 국민들과 즐겁게 소통하고 ...
                             ...                        
157    서로 믿고 의지하며, 방역에서 세계를 선도하는 대한민국의 자긍심을 계속 이어나가길 ...
158    재정이 경제 충격의 파고를 막는 방파제, 경제 회복을 앞당기는 마중물 역할을 해야 ...
159    오늘 비상경제회의에서 포스트 코로나 시대를 앞서가기 위한 국가프로젝트로서 한국판 뉴...
160    할머니들도 그런 마음이실 겁니다. 협성양로원 할머니들처럼 조심하는 마음들이 있다면,...
161    Sr. Presidente @JuanOrlandoH, nuestra primera ...
Name: text, Length: 162, dtype: object

In [54]:
df.text.head(10)

0    찡찡이가 입주했는데,걱정이 생겼네요. 관저 구석의 유리창문과 미닫이 한지창문 사이의...
1    오늘 이 추도식에 대통령으로 참석하겠다고 한 약속을 지킬 수 있게 해주신 것에 대해...
2    바다의 날 기념식은 여러모로 의미 깊었습니다. 14년 만에 대통령이 참석해 해양수산...
3    영국 런던에서 일어난 연쇄테러는 무고한 민간인과 어린이를 상대로 저질러진 범죄행위입...
4    트위터의 잭 도시 님 감사합니다. 앞으로도 트위터를 통해 국민들과 즐겁게 소통하고 ...
5                                                  NaN
6    6.25 전쟁은 아픈 역사입니다. 한반도 땅 대부분이 전쟁의 참상을 겪었고, 분단의...
7    G20 회의를 위해 독일에 도착했습니다. 북한 미사일 발사로 마음이 무겁습니다만 북...
8    여민관 집무실 창가에서 청와대 관람 오신 분들께 인사드렸습니다. 찡찡이와 마루 소식...
9    대선 때 모아주셨던 문재인 펀드를 오늘 상환해 드렸습니다. 국민들께서 보내주신 큰 ...
Name: text, dtype: object

In [40]:
#AttributeError: 'float' object has no attribute 'strip'
#astype = 데이터 타입을 변경
# \D = digit // replace A , B 는 a에서 b로 변경 
text = text.astype(str).str.replace('\D+', '')

In [41]:
sa = []
sentiment = []
for sentence in text:
  logits = test_sentences([sentence])
  sa.append(logits)
  sentiment.append(np.argmax(logits))

In [42]:
print(sentiment)

[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]


In [43]:
# 출력 로짓(sa)은 소프트맥스가 적용되지 않은 상태입니다. argmax로 더 높은 값의 위치를 라벨로 설정하면 됩니다.
sa

[array([[-1.2843131,  1.0331488]], dtype=float32),
 array([[ 0.47653905, -0.36577868]], dtype=float32),
 array([[0.01158134, 0.05242447]], dtype=float32),
 array([[-0.74694973,  0.7941785 ]], dtype=float32),
 array([[-1.1023093,  1.0300201]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[-1.4236441,  1.3496057]], dtype=float32),
 array([[0.01541391, 0.05207381]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[ 0.6547166, -0.450284 ]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[-1.6009713,  1.4918963]], dtype=float32),
 array([[ 0.23408598, -0.01328677]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[-1.3075902,  1.1984776]], dtype=float32),
 array([[-3.0280654,  2.6874504]], dtype=float32),
 array([[ 0.3452955 , -0.26514748]], dtype=float32),
 array([[-3.0280654,  2

In [44]:
df_senti = pd.DataFrame({'Col1': sa[:,0],'Col2':sa[:,1]})

TypeError: ignored

In [45]:
df_senti = pd.DataFrame({'Senti_analysis': sa[:162]})
df_senti

,Senti_analysis
0,"[[-1.2843131, 1.0331488]]"
1,"[[0.47653905, -0.36577868]]"
2,"[[0.011581342, 0.052424468]]"
3,"[[-0.74694973, 0.7941785]]"
4,"[[-1.1023093, 1.0300201]]"
...,...
157,"[[-0.6538811, 0.63117695]]"
158,"[[-0.17711367, 0.22683573]]"
159,"[[-1.2390541, 1.1134053]]"
160,"[[-0.08283585, 0.16965194]]"


In [46]:
df_senti.shape

(162, 1)

In [47]:
import functools
import operator

List_2D = [[1,2,3],[4,5,6],[7,8,9]] #List to be flattened

List_flat = functools.reduce(operator.iconcat, df_senti, [])

print("Original List:",df_senti)
print("Flattened List:",List_flat)

Original List:                    Senti_analysis
0       [[-1.2843131, 1.0331488]]
1     [[0.47653905, -0.36577868]]
2    [[0.011581342, 0.052424468]]
3      [[-0.74694973, 0.7941785]]
4       [[-1.1023093, 1.0300201]]
..                            ...
157    [[-0.6538811, 0.63117695]]
158   [[-0.17711367, 0.22683573]]
159     [[-1.2390541, 1.1134053]]
160   [[-0.08283585, 0.16965194]]
161    [[0.7087636, -0.46049088]]

[162 rows x 1 columns]
Flattened List: ['S', 'e', 'n', 't', 'i', '_', 'a', 'n', 'a', 'l', 'y', 's', 'i', 's']


In [48]:
b = [i[0][:] for i in sa]
print(b)
# [0, 2, 4, 6, 8]

[array([-1.2843131,  1.0331488], dtype=float32), array([ 0.47653905, -0.36577868], dtype=float32), array([0.01158134, 0.05242447], dtype=float32), array([-0.74694973,  0.7941785 ], dtype=float32), array([-1.1023093,  1.0300201], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([-1.4236441,  1.3496057], dtype=float32), array([0.01541391, 0.05207381], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([ 0.6547166, -0.450284 ], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([-1.6009713,  1.4918963], dtype=float32), array([ 0.23408598, -0.01328677], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([-1.3075902,  1.1984776], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([ 0.3452955 , -0.26514748], dtype=float32), array([-3.0280654,  2.6874504], dtype=float32), array([0.01763872, 0.05813183],

In [49]:
df_score = pd.DataFrame({'Senti': b[:162]})
df_score

,Senti
0,"[-1.2843131, 1.0331488]"
1,"[0.47653905, -0.36577868]"
2,"[0.011581342, 0.052424468]"
3,"[-0.74694973, 0.7941785]"
4,"[-1.1023093, 1.0300201]"
...,...
157,"[-0.6538811, 0.63117695]"
158,"[-0.17711367, 0.22683573]"
159,"[-1.2390541, 1.1134053]"
160,"[-0.08283585, 0.16965194]"


In [50]:
df_SA = pd.DataFrame(sentiment)
df_SA.columns = ["score"]
df_SA

,score
0,1
1,0
2,1
3,1
4,1
...,...
157,1
158,1
159,1
160,1


In [51]:
result_1 = pd.concat([df, df_score], axis=1, sort=False)
result = pd.concat([result_1, df_SA], axis=1, sort=False)
result

,Date,Pos,Neg,Tweet_date,text,favorites,retweets,Senti,score
0,2017년 5월 3주,81.60%,10.10%,2017-05-14,"찡찡이가 입주했는데,걱정이 생겼네요. 관저 구석의 유리창문과 미닫이 한지창문 사이의...",17444.250000,12855.250000,"[-1.2843131, 1.0331488]",1
1,2017년 5월 4주,84.10%,10.00%,2017-05-21,오늘 이 추도식에 대통령으로 참석하겠다고 한 약속을 지킬 수 있게 해주신 것에 대해...,16838.625000,14297.500000,"[0.47653905, -0.36577868]",0
2,2017년 5월 5주,78.10%,14.20%,2017-05-28,바다의 날 기념식은 여러모로 의미 깊었습니다. 14년 만에 대통령이 참석해 해양수산...,4277.500000,6269.000000,"[0.011581342, 0.052424468]",1
3,2017년 6월 1주,78.90%,14.70%,2017-06-04,영국 런던에서 일어난 연쇄테러는 무고한 민간인과 어린이를 상대로 저질러진 범죄행위입...,5432.250000,6121.750000,"[-0.74694973, 0.7941785]",1
4,2017년 6월 2주,75.60%,17.40%,2017-06-11,트위터의 잭 도시 님 감사합니다. 앞으로도 트위터를 통해 국민들과 즐겁게 소통하고 ...,3397.666667,4715.666667,"[-1.1023093, 1.0300201]",1
...,...,...,...,...,...,...,...,...,...
157,2020년 6월 2주,58.20%,37.00%,2020-05-17,"서로 믿고 의지하며, 방역에서 세계를 선도하는 대한민국의 자긍심을 계속 이어나가길 ...",2703.956191,5205.188571,"[-0.6538811, 0.63117695]",1
158,2020년 6월 3주,53.40%,41.80%,2020-05-24,"재정이 경제 충격의 파고를 막는 방파제, 경제 회복을 앞당기는 마중물 역할을 해야 ...",2796.750000,5618.750000,"[-0.17711367, 0.22683573]",1
159,2020년 6월 4주,53.30%,42.70%,2020-05-31,오늘 비상경제회의에서 포스트 코로나 시대를 앞서가기 위한 국가프로젝트로서 한국판 뉴...,3243.000000,5991.055555,"[-1.2390541, 1.1134053]",1
160,2020년 7월 1주,49.80%,45.50%,2020-06-07,"할머니들도 그런 마음이실 겁니다. 협성양로원 할머니들처럼 조심하는 마음들이 있다면,...",2671.166667,5200.854167,"[-0.08283585, 0.16965194]",1


In [ ]:
result.to_csv('/content/drive/My Drive/TweetsMoon/1227result_1.csv',encoding='utf-8-sig')